In [3]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 21.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 21.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 20.6 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler


Matplotlib is building the font cache; this may take a moment.


In [ ]:
# -------------------------------
# 1. Load and Process JSON Data
# -------------------------------
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return dat

In [ ]:

# -------------------------------
# 2. Tokenizer for Math & Story Problems
# -------------------------------
def preprocess_data(json_data):
    math_problems = [item['math'] for item in json_data if 'math' in item]
    story_problems = [item['story'] for item in json_data if 'story' in item]

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(math_problems + story_problems)

    math_sequences = tokenizer.texts_to_sequences(math_problems)
    story_sequences = tokenizer.texts_to_sequences(story_problems)

    max_length = max(len(seq) for seq in math_sequences + story_sequences)
    math_sequences = pad_sequences(math_sequences, maxlen=max_length, padding='post')
    story_sequences = pad_sequences(story_sequences, maxlen=max_length, padding='post')

    return math_sequences, story_sequences, tokenizer


In [ ]:

# -------------------------------
# 3. Drake's Formula for Numerical Processing
# -------------------------------
def apply_drakes_formula(data):
    """
    Applies Drake's Formula-style weighting to numerical data.
    This is just an initial formula structure that can be adjusted.
    """
    num_factors = len(data[0]) if len(data) > 0 else 1
    multiplier = 1.0 + (num_factors * 0.1)
    
    scaled_data = np.array(data) * multiplier
    normalization_factor = np.max(scaled_data)
    return scaled_data / normalization_factor

In [ ]:
# -------------------------------
# 4. Neural Network Model for Predictions
# -------------------------------
def build_model(input_dim, output_dim=1):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=64, input_length=100),  # Token embedding layer
        LSTM(64, return_sequences=True),
        LSTM(32),
        Dense(16, activation='relu'),
        Dense(output_dim, activation='sigmoid')  # Binary prediction output
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:

# -------------------------------
# 5. Training the Model
# -------------------------------
def train_model(json_file):
    json_data = load_json_data(json_file)
    math_data, story_data, tokenizer = preprocess_data(json_data)

    # Apply Drake’s Formula to math data
    math_data_scaled = apply_drakes_formula(math_data)

    # Splitting into training & validation sets
    split_idx = int(len(math_data_scaled) * 0.8)
    x_train, x_val = math_data_scaled[:split_idx], math_data_scaled[split_idx:]
    y_train, y_val = np.ones(len(x_train)), np.ones(len(x_val))  # Placeholder labels for now

    # Train model
    model = build_model(input_dim=len(tokenizer.word_index) + 1)
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

    return model, tokenizer


In [ ]:

# -------------------------------
# 6. Save & Load Model Functions
# -------------------------------
def save_model(model, tokenizer, model_path="ai_model.h5", tokenizer_path="tokenizer.json"):
    model.save(model_path)
    with open(tokenizer_path, 'w') as f:
        json.dump(tokenizer.word_index, f)

def load_saved_model(model_path="ai_model.h5", tokenizer_path="tokenizer.json"):
    model = tf.keras.models.load_model(model_path)
    with open(tokenizer_path, 'r') as f:
        word_index = json.load(f)
    tokenizer = Tokenizer()
    tokenizer.word_index = word_index
    return model, tokenizer

In [ ]:

# -------------------------------
# 7. Prediction Function
# -------------------------------
def predict(input_text, model, tokenizer):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')
    prediction = model.predict(padded_sequence)
    return "Positive" if prediction[0][0] > 0.5 else "Negative"


In [ ]:

# -------------------------------
# RUN TRAINING
# -------------------------------
if __name__ == "__main__":
    json_file = "training_data.json"  # Change this to the actual file path
    model, tokenizer = train_model(json_file)
    save_model(model, tokenizer)

    # Test prediction
    test_input = "If a train is traveling at 60mph and it departs at 3:00 PM..."
    print("Prediction:", predict(test_input, model, tokenizer))